# Adding Post-Newtonian general relativity corrections

It's easy to add post-newtonian corrections to your REBOUND simulations with REBOUNDx.  Let's start with a simulation without GR:

In [2]:
import rebound
sim = rebound.Simulation()
sim.add(m=1., hash="star") # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630, hash="planet") # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame
ps = sim.particles

sim.integrate(10.)
print("pomega = %.16f"%sim.particles[1].pomega)

pomega = 0.0000000000000000


As expected, the pericenter did not move at all.  Now let's add GR

In [3]:
import reboundx
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr")
rebx.add_force(gr)

The GR effects need you to set the speed of light in the right units

**The constants module has a set of constants in REBOUND's default units of AU, solar masses and yr/$2\pi$ (such that `G`=1).  If you want to use other units (e.g. $c \approx 173.14$ AU/day in Gauss units), you'd need to calculate `c`.**

In [4]:
from reboundx import constants
gr.params["c"] = constants.C

By default, the `gr` and `gr_potential` effects assume that the massive particle is at index 0 (with `gr_full` all particles are "sources" so this is not an issue).  If the massive particle has a different index, or you think it might move from index 0 in the particles array (e.g. due to a custom merger routine), you can attach a `gr_source` flag to it to identify it as the massive particle with:

In [5]:
ps["star"].params["gr_source"] = 1

Now we integrate as normal. We monitor the total Hamiltonian. Unlike other forces where we can  calculate a separate potential, here and with `gr_full` the forces are velocity dependent, which means the momentum is not just mv in a Hamiltonian framework. So rather than using `sim.energy`  and adding a potential, `gr_hamiltonian` calculates the full thing (classical Hamiltonian + gr). 

In [6]:
deltat = 100.
E0 = rebx.gr_hamiltonian(gr)
sim.integrate(sim.t + deltat)
Ef = rebx.gr_hamiltonian(gr)

print("pomega = %.16f"%sim.particles[1].pomega)
juliancentury = 628.33195 # in yr/2pi
arcsec = 4.8481368e-06 # in rad
print("Rate of change of pomega = %.4f [arcsec / Julian century]"% (sim.particles[1].pomega/deltat*juliancentury/arcsec))
print("Relative error on the relativistic Hamiltonian = {0}".format(abs(Ef-E0)/abs(E0)))

pomega = 0.0000332591781488
Rate of change of pomega = 43.1048 [arcsec / Julian century]
Relative error on the relativistic Hamiltonian = 1.110968364172078e-15


As expected, there was pericenter precession. The literature value is 42.98 arcsec / century.   

# Variants

Above we added the `gr` effect, but there are two other implementations `gr_potential` and `gr_full`.  Before running any serious simulations, you should read the more detailed descriptions at https://reboundx.readthedocs.io/en/latest/effects.html to see which implementation is appropriate for your application.

# Important Technical Details

The fastest integrator when there aren't close encounters is WHFast, which assumes that any forces can be derived from position-dependent potentials. People therefore often use the `gr_potential` implementation, which is an artificial position-dependent potential that reproduces the correct pericenter precession rate from the post-Newtonian corrections. This is likely the choice you are looking for for most applications, and there are no pitfalls there.

`gr_full` is the full 1-PN correction, and `gr` is the much simpler case where the central body dominates the mass in the system. Both of these can be written as velocity-dependent potentials, which means that they don't work with WHFast out of the box (but work fine with IAS15). In the REBOUNDx paper, we test integrating across the GR step using various methods. Here is a basic example of how to do that 

In [8]:
sim = rebound.Simulation()
sim.add(m=1., hash="star") # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630, hash="planet") # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame
ps = sim.particles
sim.integrator="whfast"
sim.dt = ps[1].P/20

rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr")
gr.params["c"] = constants.C

Our approach is to add an extra 'GR' operator to the WHFast splitting (see CustomSplittingIntegrationSchemes.ipynb). The operator we want is one that can take a given force, and integrate that effect across the timestep. We'll choose to integrate using RK2:

In [9]:
intf = rebx.load_operator("integrate_force")
intf.params['integrator'] = reboundx.integrators['rk2']

Now we need to tell the operator what force it should be integrating across the timestep:

In [10]:
intf.params['force'] = gr

Finally, we need to add the operator to the WHFast scheme. The WHFast scheme is

Kep($\frac{\Delta t}{2}$)Int($\Delta t$)Kep($\frac{\Delta t}{2}$).

We will add GR in a 1st order scheme (so we apply the operator for a full $\Delta t$ (dtfracion=1)):

Kep($\frac{\Delta t}{2}$)Int($\Delta t$)Kep($\frac{\Delta t}{2}$)GR($\Delta t$).

In [11]:
rebx.add_operator(intf, dtfraction=1, timing="post")

For a lot more discussion on different schemes and their error properties, see the REBOUNDx paper

In [13]:
deltat = 100.
E0 = rebx.gr_hamiltonian(gr)
sim.integrate(sim.t + deltat)
Ef = rebx.gr_hamiltonian(gr)

print("Relative error on the relativistic Hamiltonian = {0}".format(abs(Ef-E0)/abs(E0)))

Relative error on the relativistic Hamiltonian = 9.572743845114997e-08
